In [9]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, LSTM, Dense, TimeDistributed, Dropout, BatchNormalization, Conv1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import initializers

from scripts.layers import Spectrogram, MelSpectrogram, LogMelSpectrogram
from scripts.metrics import Precision, Recall, F1

import json
import os

In [13]:
# preprocessing parameters
params = {'sample_rate': 22050,
          'n_fft': 1000,
          'hop_length': 256,
          'frame_size': 256,  # ??
          'n_mels': 64,
          'min_freq': 0,
          'max_freq': 8000,
          'x_max': None,
          'x_min': None,
          'data_mean': None,
          'data_std': None}

with open('stats/params.json', 'w') as json_file:
    json.dump(params, json_file)

# model hyper parameters
hparams = {'num_conv_filters': 32,
           'conv_kernel_size': 1,
           'conv_stride': 2,
           'gru_units': 128,
           'lr': 0.001,
           'loss_function': 'binary_crossentropy',
           'epochs': 100}

with open('stats/hparams.json', 'w') as json_file:
    json.dump(hparams, json_file)

# optimizer
optimizer = Adam(learning_rate=hparams['lr'])

# metrics
metrics = [
    Precision,
    Recall,
    F1
]

# input
model = Sequential(name='Trigger-word-Marvin-model')
model.add(tf.keras.layers.Input(shape=(params['sample_rate'],),
                                dtype=tf.float32))

# preprocessing
preprocessing = Sequential([
    Spectrogram(params, hparams),
    MelSpectrogram(params, hparams),
    LogMelSpectrogram(params, hparams),
], name='preprocessing_layer')
model.add(preprocessing)

# convolutional units
conv = Sequential([
    Conv1D(hparams['num_conv_filters'],
           hparams['conv_kernel_size'],
           hparams['conv_stride'])
], name='conv_layer')
model.add(conv)

# recurrent units
rnn = Sequential([
    GRU(hparams['gru_units'], return_sequences=True),
    BatchNormalization(),
    GRU(hparams['gru_units'], return_sequences=True),
    BatchNormalization(),
    GRU(hparams['gru_units'], return_sequences=True),
    BatchNormalization(),
    GRU(hparams['gru_units'], return_sequences=False)
], name='rnn_layer')
model.add(rnn)

# dense units
dense = Sequential([
    Dense(1, activation='sigmoid',
          bias_initializer=initializers.Constant(-2.197))
], name='dense_layer')
model.add(dense)

# compile model
model.compile(
    optimizer=optimizer,
    loss=hparams['loss_function'],
    metrics=metrics
    )

print(model.summary())

Model: "Trigger-word-Marvin-model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 preprocessing_layer (Sequen  (None, 86, 64)           0         
 tial)                                                           
                                                                 
 conv_layer (Sequential)     (None, 43, 32)            2080      
                                                                 
 rnn_layer (Sequential)      (None, 128)               360960    
                                                                 
 dense_layer (Sequential)    (None, 1)                 129       
                                                                 
Total params: 363,169
Trainable params: 362,401
Non-trainable params: 768
_________________________________________________________________
None


In [12]:
# save model
model_dir = 'models/'
if not os.path.isdir(model_dir):
        os.mkdir(model_dir)

# serialize model to json
model_json = model.to_json()
with open(model_dir + 'marvin-model-struct.json', 'w') as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save_weights(model_dir + 'marvin-model-init-weights.h5')
